### **Definir el directorio de trabajo**
Define el directorio de trabajo como la carpeta `data` de la carpeta compartida `DS4A-Team12` de Drive.
Este directorio debe contener el archivo de datos original `Sociodemografico.zip` (5945450 registros, 140 MB) en el directorio `raw_data` y el diccionario de datos `dtypes_sociodemo.json` en el directorio `data_dictionaries`.


In [ ]:
import os
import sys
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/ICBF/data


In [ ]:
# Librerias relevantes
import time
import pandas as pd
import numpy as np
import json
import re
from utils import replace_log
from utils import impute_nans, impute_years_log, impute_nans_range_log


### **Lectura del archivo**

In [ ]:
# Abre el diccionario con la estructura de datos definida
with open('datatypes_dictionaries/dtypes_sociodemo.json', 'r') as file:
  dtypes_soc = json.load(file)
# Diccionario con categorías válidas (ordenadas) para cada variable categórica (ordinal)
with open('datatypes_dictionaries/soc_cat.json', 'r') as file:
  soc_cat = json.load(file)
# Lee el archivo
soc = pd.read_parquet('preprocessed_data/sociodemo.parquet')
soc = soc.astype(dtypes_soc)
soc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5945350 entries, 0 to 5945349
Data columns (total 68 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   Cod_clase                  category      
 1   Ind_grupo_sisben_4         category      
 2   Ind_nivel_sisben_4         category      
 3   Tip_vivienda               category      
 4   Ind_tiene_energia          category      
 5   Ind_tiene_alcantarillado   category      
 6   Ind_tiene_gas              category      
 7   Ind_tiene_recoleccion      category      
 8   Ind_tiene_acueducto        category      
 9   n_hogares_vivienda         UInt8         
 10  Tip_ocupa_vivienda         category      
 11  Ind_tiene_cocina           category      
 12  Ind_tiene_nevera           category      
 13  Ind_tiene_lavadora         category      
 14  Ind_tiene_pc               category      
 15  Ind_tiene_internet         category      
 16  n_personas_hogar           UInt8    

In [ ]:
# Lista de variables categóricas ordinales
cols_cat = [col for col in soc.columns if col in soc_cat.keys()]
# Lista de variables categóricas ordinales
cols_ord = ['Ind_grupo_sisben_4', 'Ind_nivel_sisben_4', 
            'Nivel_educ_padre', 'Nivel_educ_madre', 
            'anno_encuesta', 'Estrato']
cols_fecha = [col for col in soc.columns if col[:5] == 'Fecha']
cols_medidas = ['MedicionPerimetroBraquial', 'Peso', 'Talla']
cols_zscore = [col for col in soc.columns if col[:6] == 'ZScore']
cols_estado = [col for col in soc.columns if col[:6] == 'Estado']
cols_zscore_peso = [col for col in soc.columns if col[:10] == 'ZScorePeso']
cols_estado_peso = [col for col in soc.columns if col[:10] == 'EstadoPeso']

In [ ]:
# Forzar categorías y ordenar las categorías 
for col in cols_cat:
  categorias = soc_cat[col]
  ordinal = col in cols_ord
  soc[col] = soc[col].cat.set_categories(categorias, ordered = ordinal)

In [ ]:
for col in soc.columns:
  print(f"**{col}** {soc[col].unique()[:5]}")

**Cod_clase** [1, 2, 3]
Categories (3, int64): [1, 2, 3]
**Ind_grupo_sisben_4** ['A', 'B', 'C', 'D']
Categories (4, object): ['A' < 'B' < 'C' < 'D']
**Ind_nivel_sisben_4** [3, 5, 2, 4, 1]
Categories (51, int64): [1 < 2 < 3 < 4 ... 48 < 49 < 50 < 51]
**Tip_vivienda** [2, 1, 3, 4, 5]
Categories (5, int64): [2, 1, 3, 4, 5]
**Ind_tiene_energia** [1, 0]
Categories (2, int64): [1, 0]
**Ind_tiene_alcantarillado** [1, 0]
Categories (2, int64): [1, 0]
**Ind_tiene_gas** [1, 0]
Categories (2, int64): [1, 0]
**Ind_tiene_recoleccion** [1, 0]
Categories (2, int64): [1, 0]
**Ind_tiene_acueducto** [1, 0]
Categories (2, int64): [1, 0]
**n_hogares_vivienda** <IntegerArray>
[1, 2, 3, 4, 7]
Length: 5, dtype: UInt8
**Tip_ocupa_vivienda** [1, 4, 5, 3, 2]
Categories (5, int64): [1, 4, 5, 3, 2]
**Ind_tiene_cocina** [1, 0]
Categories (2, int64): [1, 0]
**Ind_tiene_nevera** [1, 0]
Categories (2, int64): [1, 0]
**Ind_tiene_lavadora** [0, 1]
Categories (2, int64): [0, 1]
**Ind_tiene_pc** [0, 1]
Categories (2, int

In [ ]:
soc.to_parquet('clean_data/sociodemo_filtered.parquet')

In [ ]:
# Escribe un archivo con los cambios generados
with open('log_files/log_sociodemo_1.json', 'w') as log_file:
  log_file.write('Preprocesamiento_Sociodemográfico:\n')
  for line in log_fechas:
    log_file.write('\t' + line + '\n')
 
# Guardar el dataframe preprocesado en un archivo parquet
soc.to_parquet('preprocessed_data/sociodemo.parquet')
print(f'Nuevo archivo guardado: {time.time() - t0:.2f} s.')